# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Embark on a transformative journey with DeepLearning.AI, a premier platform reshaping the landscape of artificial intelligence. Offering cutting-edge courses curated by the world-renowned expert Andrew Ng, DeepLearning.AI equips learners with the skills to thrive in the AI-driven world. Whether you aim to delve into Neural Networks, Machine Learning, or Natural Language Processing, this platform caters to all. Elevate your understanding of AI technologies and unlock exciting career opportunities. Join DeepLearning.AI today and unleash your potential in this dynamic field. Dive deep into the realm of AI and let your knowledge flourish with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Embark on a transformative journey with DeepLearning.AI, a premier platform reshaping the landscape of artificial intelligence. Offering cutting-edge courses curated by the world-renowned expert Andrew Ng, DeepLearning.AI equips learners with the skills to thrive in the AI-driven world. Whether you aim to delve into Neural Networks, Machine Learning, or Natural Language Processing, this platform caters to all. Elevate your understanding of AI technologies and unlock exciting career opportunities. Join DeepLearning.AI today and unleash your potential in this dynamic field. Dive deep into the realm of AI and let your knowledge flourish with DeepLearning.AI.

----------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Embark on a transformative journey with DeepLearning.AI, a premier platform reshaping the landscape of artificial intelligence. Offering cutting-edge courses curated by the world-renowned expert Andrew Ng, DeepLearning.AI equips learners with the skills to thrive in the AI-driven world. Whether you aim to delve into Neural Networks, Machine Learning, or Natural Language Processing, this platform caters to all. Elevate your understanding of AI technologies and unlock exciting career opportunities. Join DeepLearning.AI today and unleash your potential in this dynamic field. Dive deep into the realm of AI and let your knowledge flourish with DeepLearning.AI.

----------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

After aggregating feedback from all reviewers, here are the key points to consider for improving the writing:

1. **SEO Optimization**:
   - Include target keywords, optimize meta tags, and implement internal linking strategies to enhance SEO performance. This will improve search engine visibility and user engagement.

2. **Accuracy and Verifiability**:
   - Ensure all claims made about DeepLearning.AI, Andrew Ng, and the platform's offerings are accurate and backed up by verifiable information. Avoid exaggerated or unsubstantiated statements to prevent misleading consumers.

3. **Intellectual Property Concerns**:
   - Review the content for any potential intellectual property issues, such as trademark infringement or unauthorized use of copyrighted material related to AI technologies. Ensure that the content does not infringe on any trademarks or copyrighted material.

4. **Et

## Get the summary

In [15]:
print(res.summary)

Title: Unleashing the Potential of DeepLearning.AI: A Comprehensive Review

Embark on an enriching journey with DeepLearning.AI, a leading platform revolutionizing the realm of artificial intelligence. Developed by the esteemed expert Andrew Ng, this platform offers a wealth of transformative courses designed to empower learners in the AI domain. Explore an array of topics from Neural Networks to Natural Language Processing, honing your skills for a future-driven career. Dive deep into the world of AI with confidence, as DeepLearning.AI equips you with the knowledge needed to excel. Join today and embark on a learning experience that will reshape your future in AI.
